### Get data from quandl

In [86]:
import quandl
import pandas as pd
import numpy as np

Show the data

In [87]:
quandl.api_config = "BxM9-HUoVqZqzxKwwWm1"
df = pd.read_csv('Googl.csv')
df.columns

Index(['Date', 'Adj. Open', 'Adj. High', 'Adj. Low', 'Adj. Close',
       'Adj. Volume'],
      dtype='object')

### Keep important columns

In [88]:
df = df[['Date','Adj. Open','Adj. High','Adj. Low','Adj. Close','Adj. Volume']]

In [89]:
# df.to_csv('Googl.csv', sep=',', encoding='utf-8')  # Save the data to local

### Add percent volatility/percent change

In [90]:
df['HL_PCT'] = (df['Adj. High'] - df['Adj. Close']) / df['Adj. Close'] * 100.0

In [91]:
df['PCT_change'] = (df['Adj. Close'] - df['Adj. Open']) / df['Adj. Open'] * 100.0 
# Daily percent change
df = df[['Date','Adj. Close', 'HL_PCT', 'PCT_change', 'Adj. Volume']] # Columns we care about
df.head()

,Date,Adj. Close,HL_PCT,PCT_change,Adj. Volume
0,2004-08-19,50.322842,3.712563,0.324968,44659000.0
1,2004-08-20,54.322689,0.710922,7.227007,22834300.0
2,2004-08-23,54.869377,3.729433,-1.227880,18256100.0
3,2004-08-24,52.597363,6.417469,-5.726357,15247300.0
4,2004-08-25,53.164113,1.886792,1.183658,9188600.0


In [92]:
forecast_col = 'Adj. Close'
df.fillna(-99999, inplace=True)  # Fill NaN value with -99.999

In [93]:
import math
forecast_out = int(math.ceil(0.01 * len(df)))

In [94]:
forecast_out

35

Create label column

In [95]:
df['label'] = df[forecast_col].shift(-forecast_out)  # Shift label column up *forecast_out days

In [96]:
# df.dropna(inplace=True)
df.head()

,Date,Adj. Close,HL_PCT,PCT_change,Adj. Volume,label
0,2004-08-19,50.322842,3.712563,0.324968,44659000.0,69.078238
1,2004-08-20,54.322689,0.710922,7.227007,22834300.0,67.839414
2,2004-08-23,54.869377,3.729433,-1.227880,18256100.0,68.912727
3,2004-08-24,52.597363,6.417469,-5.726357,15247300.0,70.668146
4,2004-08-25,53.164113,1.886792,1.183658,9188600.0,71.219849


### Import scikit models

In [97]:
from sklearn import preprocessing, model_selection, svm
from sklearn.linear_model import LinearRegression

### Using model

In [98]:
X = np.array(df.drop(['label'],1))

X = preprocessing.scale(X)
df.dropna(inplace=True)
y = np.array(df['label'])
X_lately = X[-forecast_out:]

X = X[:-forecast_out]


len(X), len(y), len(df)

ValueError: could not convert string to float: '2004-08-19'

In [ ]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.2)

In [ ]:
# Create classifier using LinearRegression
clf = LinearRegression(n_jobs=-1)
clf.fit(X_train,y_train)
accuracy = clf.score(X_test,y_test)
accuracy

### Predict y

In [ ]:
forecast_set = clf.predict(X_lately)
print(forecast_set)
print(accuracy)
print(forecast_out)

### Graphing the data

In [ ]:
import datetime
import matplotlib.pyplot as plt
from matplotlib import style
style = 'ggplot'

# Create empty forecast column
df['Forecast'] = np.nan

# Create datetime
last_date = df.iloc[-1].name
# last_unix = last_date.timestamp()
# one_day = 86_400
# next_unix = last_unix + one_day
df